## Vector Store and Retrivers
#### Langchain's Vector Store and Retriever abstractions.

These abstractions are designed to support retrieval of data from vector databases and other sources -- for integration with LLM workflows.

They are important for applications that fetch data to be reasoned over part of model inference, as in the case of RAG (Retrieval-Augmented Generation)

We will cover:

* Documents
* Vector stores
* Retrievers

### Documents

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;

- metadata: a dict containing arbitrary metadata.

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [33]:
from langchain_core.documents import Document

documents = [
    Document(
    page_content="Dogs are great companions, known for their loyalty and friendliness.",
    metadata={"source": "mammal-pets-doc"},
    ),

    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),

    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),

    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),

    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [34]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [35]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002AA6EE66D50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002AA6EE67750>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [37]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings=GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002AA6E1BC050>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002AA6EF6C190>, model='models/gemini-embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [38]:
# Vector store
from langchain_chroma import Chroma

vectoreStore = Chroma.from_documents(documents, embeddings)
vectoreStore

In [39]:
vectoreStore.similarity_search("cat")

[Document(id='4f52a29c-9b60-4c94-a3ef-be63aafa04d7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='5f1d4e17-76cf-4dbe-8813-843daa3593de', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='d9fd9156-4bea-4ea8-b95f-b04b099cf5ed', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='5f01a6fe-b745-4784-9007-8a112f36728c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [40]:
# Async query
await vectoreStore.asimilarity_search("cat")

[Document(id='4f52a29c-9b60-4c94-a3ef-be63aafa04d7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='5f1d4e17-76cf-4dbe-8813-843daa3593de', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='d9fd9156-4bea-4ea8-b95f-b04b099cf5ed', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='5f01a6fe-b745-4784-9007-8a112f36728c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [41]:
vectoreStore.similarity_search_with_score("cat")

[(Document(id='4f52a29c-9b60-4c94-a3ef-be63aafa04d7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.3450752794742584),
 (Document(id='5f1d4e17-76cf-4dbe-8813-843daa3593de', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.3450752794742584),
 (Document(id='d9fd9156-4bea-4ea8-b95f-b04b099cf5ed', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.5362598896026611),
 (Document(id='5f01a6fe-b745-4784-9007-8a112f36728c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.5362598896026611)]

### Retrievers

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.


We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [42]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectoreStore.similarity_search).bind(k=1)
retriever.batch(['cat', 'dog'])

[[Document(id='5f1d4e17-76cf-4dbe-8813-843daa3593de', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='d9fd9156-4bea-4ea8-b95f-b04b099cf5ed', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStore Retriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [43]:
retriever = vectoreStore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 1}
)

In [44]:
retriever.batch(['cat', 'dog'])

[[Document(id='5f1d4e17-76cf-4dbe-8813-843daa3593de', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='d9fd9156-4bea-4ea8-b95f-b04b099cf5ed', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

#### Integrate Retriever with chain

In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using provided context and if you don't know the answer, just say that you don't know, don't try to make up an answer.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([('human', message)])

rag_chain = {'context': retriever, 'question': RunnablePassthrough()} | prompt | model

response = rag_chain.invoke('Tell me something about cats?')
response

AIMessage(content='Cats are independent pets that often enjoy their own space.', additional_kwargs={'reasoning_content': 'We need to answer the question "Tell me something about cats?" using provided context. The context says: "Cats are independent pets that often enjoy their own space." So we can answer that. Ensure we don\'t fabricate beyond that.'}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 164, 'total_tokens': 231, 'completion_time': 0.132978351, 'prompt_time': 0.006534503, 'queue_time': 0.047661277, 'total_time': 0.139512854}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bf05c712-10d6-41fc-b6f6-c7f74fddb376-0', usage_metadata={'input_tokens': 164, 'output_tokens': 67, 'total_tokens': 231})